# Presenting predictions

In [13]:
#imports 
import sys
sys.path.append('../src')

import models.model as demo

In [14]:
PROJECT_PATH = '/home/tomasmizera/school/nsiete/nsiete-project/project/'

First, we create model

In [15]:
model = demo.create_model()

Second, load saved model

In [4]:
model.load_weights(PROJECT_PATH + 'models/model.ckpt')

Third, get data generator

In [5]:
data_generator = demo.train_generator

Fourth, we get sample data, store true labels and prepare X for prediction

In [6]:
X, Y = data_generator.__getitem__(0)

In [7]:
X.shape

(32, 256, 384, 3)

In [8]:
Y.shape

(32, 256, 384, 4)

**X** - consists of images (batch size * (resolution) * channels)  
**Y** - masks for every cloud type (batch size * (resolution) * num of cloud classes)

In [9]:
# one pixel for each channel -- color
X[0][0][0]

array([0.3546837 , 0.37036997, 0.37612975])

Next, predict mask for input image X

In [12]:
data_generator.latest_images

['/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//9ba0225.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//ff00e81.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//9a9d599.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//1dacc16.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//72e4b6e.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//3fc1779.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//7adf177.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//b9425c3.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//e5405df.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//59f6a74.jpg',
 '/home/tomasmizera/school/nsiete/nsiete-project/project/data/train_images//e317214.jpg',
 '/home/to